In [1]:
import torch
from torch.autograd import Variable
import torchvision

import numpy as np
import os, time

from utilities.data import CIFAR10, MNIST, SVHN
from utilities.init import make_dirs, init_data_loader, write_preprocessor
from utilities.output import write_logger, write_observations
from utilities.parser import get_default_parser, update_code_dim
from architecture.nn import VAE

to_np = lambda x: x.data.cpu().numpy()
normalize_to_zero_one = lambda x: (x + 1.) / 2.
torch.cuda.set_device("cuda:4")

In [2]:
torch.backends.cudnn.benchmark = True

parser = get_default_parser()
config = parser.parse_args(args=[])

make_dirs(config.ckpt_path, config.data_path, config.img_path)

data_loader, config.img_size, config.num_channels = init_data_loader(config.dataset, config.data_path, config.batch_size, training_digits=config.training_digits)

update_code_dim(config)
write_preprocessor(config)

v = VAE(config)
v = v.cuda()
v.train()

if config.ckpt:
    v.load_state_dict(torch.load(os.path.join(config.ckpt_path, 'v.pth')))

v_optim = torch.optim.Adam(v.parameters(), lr=config.lr_adam, betas=(config.beta_1, config.beta_2))
bce_loss = torch.nn.BCELoss(size_average=False) # no averaging, sum over batch, height and width

logger = {"loss": np.array([])}
z_fixed = Variable(torch.randn(config.num_samples, config.z_dim)).type(torch.cuda.FloatTensor)

{'batch_size': 32, 'num_epochs': 100, 'num_samples': 64, 'num_every_nth_epoch': 1, 'num_gpus': 1, 'num_workers': 0, 'ckpt': False, 'ckpt_path': 'ckpt', 'data_path': 'data', 'img_path': 'img', 'output_path': '', 'dataset': 'mnist', 'training_digits': 0, 'z_dim': 10, 'lr_adam': 0.001, 'beta_1': 0.5, 'beta_2': 0.999, 'std': 0.02, 'num_filters_in_final_layer': 128, 'num_conv_layers': 4, 'model': 'vae', 'img_size': 64, 'num_channels': 1, 'c_dim': [128, 4, 4]}


/home/danliwoo/.conda/envs/beer/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [ ]:
for epoch in range(config.num_epochs):
    if epoch >= 1:
        print("\n[%2.2f]" % (time.time() - t0), end="\n")
    t0 = time.time()

    for step, (images, _) in enumerate(data_loader, 0):
        if step >= 1:
            print("[%i] [%i] [%2.2f] [%2.2f]" % (epoch, step, time.time() - t1, to_np(loss)), end="\r")
        t1 = time.time()

        batch_size = images.size(0)

        x = Variable(images.type(torch.cuda.FloatTensor))
        x = normalize_to_zero_one(x)
        x_r = v(x) # reconstruction

        loss_r = bce_loss(x_r, x) / batch_size
        loss_kl = torch.mean(.5 * torch.sum((v.mu**2) + torch.exp(v.log_sigma_sq) - 1 - v.log_sigma_sq, 1))
        loss = loss_r + loss_kl
        loss.backward()

        v_optim.step()
        v_optim.zero_grad()

    if epoch % config.num_every_nth_epoch == 0:
        write_observations(config, epoch, z_fixed, v, logger, loss)

/home/danliwoo/.conda/envs/beer/lib/python3.7/site-packages/torch/tensor.py:314: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


[0] [185] [0.02] [750.51]
[5.33]
[1] [185] [0.02] [641.25]
[5.18]
[2] [185] [0.02] [660.69]
[5.19]
[3] [185] [0.02] [611.75]
[4.96]
[4] [185] [0.02] [614.11]
[5.08]
[5] [185] [0.02] [607.18]
[5.04]
[6] [185] [0.02] [616.63]
[5.00]
[7] [185] [0.01] [598.55]
[4.80]
[8] [185] [0.02] [579.65]
[4.51]
[9] [185] [0.02] [585.81]
[5.29]
[10] [185] [0.02] [575.53]
[5.19]
[11] [185] [0.02] [563.08]
[5.62]
[12] [185] [0.02] [587.58]
[5.08]
[13] [185] [0.02] [574.71]
[4.89]
[14] [185] [0.02] [584.81]
[4.70]
[15] [185] [0.02] [575.29]
[4.88]
[16] [185] [0.02] [595.90]
[5.01]
[17] [185] [0.02] [565.49]
[4.98]
[18] [185] [0.02] [566.31]
[5.05]
[19] [185] [0.02] [582.49]
[5.26]
[20] [185] [0.02] [576.12]
[4.79]
[21] [185] [0.02] [576.39]
[4.79]
[22] [185] [0.02] [574.60]
[4.96]
[23] [185] [0.01] [570.35]
[4.77]
[24] [185] [0.02] [545.28]
[4.94]
[25] [185] [0.02] [567.63]
[4.95]
[26] [185] [0.02] [567.66]
[5.29]


In [ ]:
write_logger(config, logger)

Best path is stored at `ckpt/v.pth`